In [149]:
import pandas as pd
import numpy as np
from tabulate import tabulate

In [151]:
# resource = 'https://drive.google.com/uc?export=download&id=1by5_2HhJ4WO5VBJ0Tc9OZStPqDwiAUYW'
resource = 'https://drive.google.com/uc?export=download&id=15tURO14aosIqB2pIc-fJQ_pgHAPM8Ho-'
dataset = pd.read_csv(resource)

## Definir Ponderacion
___

- Ponderaciones asignadas al programa académico de Ingeniería de Sistemas (UIS)
  
|Ciencias Naturales|Lectura Crítica|Matemáticas|Sociales y Ciudadanias|Inglés|
|--|--|--|--|--|
|25%|20%|35%|10%|10%| 

## Proceso de Selección
___

- Multiplicacion del ponderado asignado a cada una de las áreas del ICFES 
   de la universidad por el valor de los resultados del Exámen ICFES Saber 11


In [152]:
def selection_process(dataset):
    # Define ponderation
    nature_sciences = 0.25
    critical_reading = 0.2
    maths = 0.35
    sociology = 0.1
    english = 0.1

    # create sub_dataset 
    columns=['punt_lectura_critica',
            'punt_matematicas',
            'punt_c_naturales',
            'punt_sociales_ciudadanas',
            'punt_ingles',
            'punt_global']

    df = pd.DataFrame(data=dataset,index=None,columns=columns)

    # transform_columns

    df['punt_lectura_critica'] = df['punt_lectura_critica'] * critical_reading
    df['punt_matematicas'] = df['punt_matematicas'] * maths
    df['punt_c_naturales'] = df['punt_c_naturales'] * nature_sciences
    df['punt_sociales_ciudadanas'] = df['punt_sociales_ciudadanas'] * sociology
    df['punt_ingles'] = df['punt_ingles'] * english

    # calculate punt_ponderate
    df['punt_ponderado'] = df.iloc[:,:5].sum(axis=1)

    # move the last column to penult position
    df_cols = df.columns.tolist()
    reorder = columns[:-1] + df_cols[-1:]
    reorder.append(columns[-1])
    reorder

    df = df[reorder]
    return df.sort_values(by=['punt_ponderado','punt_global'], ascending=False)

### Porcentaje de Selección
___
- Para seleccionar un top de 5 puestos, se define:

|Puesto|%Ponderado|
|--|--|
|1|> 60%|
|2|[55% - 60%)|
|3|[45% - 55%)|
|4|[35% - 45%)|
|5|< 35%|


#

In [157]:
# conditions
def define_top(df):
    df_positions = df.copy()
    df_positions['puesto'] = np.where(df_positions.punt_ponderado>60,1,
                                np.where((df_positions.punt_ponderado>=55) & (df_positions.punt_ponderado<60),2,
                                np.where((df_positions.punt_ponderado>=45) & (df_positions.punt_ponderado<55),3,
                                np.where((df_positions.punt_ponderado>=35) & (df_positions.punt_ponderado<45),4,
                                np.where(df_positions.punt_ponderado<35,5,0)))))
    return df_positions.sort_values(by=['punt_ponderado','punt_global'], ascending=False)

df = selection_process(dataset)
df_positions= define_top(df)

df_positions.shape

(15435, 8)